In [ ]:
#
# [TODO]  PCA 수행시 데이터 표준화 → 공분산 행렬 → 공분산 행렬의 고유벡터를 구하고 나서
# 1) 고유벡터에 표준화 데이터를 투영시킨 것이 주성분인데, 이 과정이 올바른 것인지 명확치 않음 
#    : sklearn.decomposition.PCA 에 표준화 데이터를 넣어 주성분을 구한 것과 
#      위 과정을 manual 하게 수행한 결과에 표준화 데이터를 투영한 것의 결과가 서로 다름 
#    : maybe ㅇnumpy 행렬 계산시 행, 열 기준축 때문인지에 대해서 등 원인을 파악하지 못함
# 2) Singular Vector Descomposition 을 통해 주성분을 구하는 로직을 파악하지 못함
# 

In [31]:
#
# [1] 분산 (Variance)
#
# Var ( X ) = E [ ( X - E[X] )² ]
#
# → E[ ( X - E[X] )² ]
# → E[ X² - 2 X · E[X] + E[X]² ]
# → E[X²] - E[2X· E[X]] + E[E[X]²]
# → E[X²] - 2E[X]E[X] + E[X]²
# → E[X²] - E[X]²
#
# - 변수가 변화하는 정도

import numpy as np
import pandas as pd

X = np.array([1, 2, 3, 4, 5])

m = np.mean(X)
n = len(X)

print(np.sum((X - m) ** 2) / n , np.mean(X*X) - np.mean(X)**2 , np.var(X))

# 
# 표본은 자유도 (n - 1) 고려
# 
df = pd.DataFrame()
df["X"] = X
np.sum((X - m)**2) / ( n - 1) ,  df["X"].var()

2.0 2.0 2.0


(2.5, -0.3125, 2.5)

In [28]:
#
# [2] 공분산 (Covariance)
#
# Cov (X, Y) = E [ ( X - E[X] ) * ( Y - E[Y] ) ] = E[X·Y] - E[X]·E[Y]
# 
# → E[(X - E[X]) * (Y - E[Y])]
# → E[ X·Y - E[X]·Y - E[Y]·X + E[X]E[Y] ]
# → E[X·Y] - E[E[X]·Y] - E[E[Y]·X] + E[E[X]E[Y]]
# → E[X·Y] - E[X]·E[Y] - E[Y]·E[X] + E[X]E[Y]
# → E[X·Y] - E[X]·E[Y]              * X, Y 독립인 경우 E[XY] = E(X)E(Y) 이므로 공분산 0
#
# - 두 변수가 함께 변하는 정도 (같은 방향으로 변하면 양수, 반대 방향으로 변하면 음수)
#
import numpy as np
import pandas as pd

X1 = np.array([1, 2, 3, 4, 5])
X2 = np.array([2, 4, 6, 8, 10])

# 방법 1 : E[(X - E[X]) * (Y - E[Y])] 
np.mean((X1 - np.mean(X1)) * (X2 - np.mean(X2)))  # 4.0

# 방법 2 : E[X·Y] - E[X]·E[Y]
np.mean(X1 * X2) - np.mean(X1) * np.mean(X2)      # 4.0

# 방법 3: Covariance Matrix 에서 추출
np.cov(X1, X2, ddof=0) [0, 1]                     # 4.0

# 방법 4
(X1 - np.mean(X1))@(X2 - np.mean(X2)) / len(X1)   # 4.0

4.0

In [52]:
#
# [4] 선형 변환 時 기대값, 분산, 표준편차, 공분산의 성질
#
# - E(aX + b) = aE(X) + b
# - V(aX + b) = a^2 * V(X)
# - SD(aX + b) = |a| * SD(X)
# - Cov(aX + b, cY + d) = ac * Cov(X, Y)
#
import numpy as np

X = np.array([1,2,3,4,5])
Y = np.array([2,4,6,8,10])
a = 10
b = 5
c = 3
d = 6

# E(aX+b) = a * E(X) + b
E_X = np.mean(X)
np.mean(a * X + b) == a * E_X + b      # True

# V(aX+b) = a^2 * V(X)
V_X = np.var(X)
np.var(a * X + b) == a ** 2 * V_X      # True

# S(aX+b) = |a| * S(X) 
SD_X = np.std(X)
np.std(a * X + b) == a * SD_X          # True
a = -10 
b = -5
np.std(a * X + b) == np.abs(a) * SD_X  # True

# Cov(aX + b, cY + d) = a * c * Cov(X, Y)
C_XY = np.cov(X, Y, ddof = 0)[0,1] 
np.cov(a*X + b, c*Y + d, ddof = 0)[0,1] == a * c * C_XY  # True

True

In [70]:
#
# [6] 선형 결합 時 기대값, 분산, 표준편차, 공분산의 성질
# 
# - E(aX + bY) = aE(X) + bE(Y)
# - V(aX + bY) = a^2 * V(X) + b^2 * V(Y) + 2ab * Cov(X, Y)
# - SD(aX + bY) = sqrt(a^2 * Var(X) + b^2 * Var(Y) + 2ab * Cov(X, Y))
# - Cov(aX + bY, cX + dY) = ac * Cov(X, X) + ad * Cov(X, Y) + bc * Cov(Y, X) + bd * Cov(Y, Y)
#
import numpy as np

X = np.array([1,2,3,4,5])
Y = np.array([2,4,6,8,10])
a = 10
b = 5
c = 3
d = 6

E_X = np.mean(X)
E_Y = np.mean(Y)
V_X = np.var(X)
V_Y = np.var(Y)
S_X = np.std(X)
S_Y = np.std(Y)
C_XY = np.cov(X,Y, ddof=0)[0,1]  # 2.0
C_XX = np.cov(X,X, ddof=0)[0,1]  # 4.0
C_YX = np.cov(Y,X, ddof=0)[0,1]  # 4.0
C_YY = np.cov(Y,Y, ddof=0)[0,1]  # 8.0

np.mean(a*X + b*Y) == a * E_X + b * E_Y
np.var(a*X + b*Y) == a**2 * V_X + b**2*V_Y + 2*a*b*C_XY 
np.std(a*X + b*Y) == np.sqrt(a**2 * V_X + b**2*V_Y + 2*a*b*C_XY)
np.cov(a*X+b*Y, c*X+d*Y, ddof=0)[0,1] == a*c*C_XX + a*d*C_XY + b*c*C_YX + b*d*C_YY

True

In [ ]:
#
# [참고] 확률변수의 곱 
#
# - E(XY) = E(X)E(Y) * X, Y 가 독립인 경우
# - 분산, 공분산의 경우는 상호작용에 따라 달라지는 부분이 커 정의할 수 없음
#

In [ ]:
#
# [7] 상관 계수의 성질
# 
# - 두 변수의 선형 변환 時 상관계수의 크기는 변화 없음 
# - 두 변수 중 하나에 음수가 곱해지면 상관계수의 부호가 변화
# 
# 예) A, B 의 상관계수가 1/2 이고 A = 10x + 5, B = 20y + 8 인 경우 x , y 의 상관계수는 ?
#
# 1) p(A, B) = Cov (A, B) / σ_A * σ_B = 1 / 2
#    →  Cov (A, B) = p (A, B) * σ_A * σ_B = 1 / 2 * σ_A * σ_B
#
# 2) 
# σ_A = σ (10X + 5) = 10 * σ_x
# σ_B = σ (20y + 8) = 20 * σ_y
#
# 3) Cov (A, B) = Cov (10x + 5, 20y + 8 ) = 10 * 20 * Cov (x , y) = 200 * Cov (x, y)
# 
# 4)
# 1) ← 2) & 3)
# Cov (A, B) = 200 * Cov (x, y) = 1 / 2 * σ_A * σ_B = 1 / 2 * (10 * σ_x) * (20 * σ_y)
# → 200 Cov (x y) = 100 * σ_x * σ_y
#
# ▷ Cov (x, y) / σ_x * σ_y = 100 / 200 = 1 / 2
#

In [92]:
#
# [8] 공분산 행렬 (Covariance Matrix)
# 
# Cov (X) = E ( X - E[X] ) · ( X - E(X) )_T  / n  * Transepose 는 X 행렬 형태에 의존
#  
# - 대각 행렬은 各 변수의 분산, 나머지 요소가 변수 間 공분산 
# - ddof 로 자유도 설정 , rowvar 로 데이터의 구성 방향(행 방향이 default) 설정
# 
#
# [NOTE] X_T · X 와 X_T · X 
# - X (n x p) dot X_T (p x n) 은 n x n 으로 n 에 대한 정리
# - X_T (p x n) dot X (n x p) 은 p x p 으로 p 에 대한 정리
#

import numpy as np

X1 = np.array([1, 2, 3, 4, 5])
X2 = np.array([2, 4, 6, 8, 10])

X = np.stack([X1, X2], axis=1)   # p x n 
C1 = (X - np.mean(X, axis = 0)).T @ (X - np.mean(X, axis = 0)) / len(X)
C2 = np.cov(X1, X2, ddof=0)
C3 = np.cov(X.T, ddof=0)         # numpy 는 기본적으로 행
C1, C2, C3

(array([[2., 4.],
        [4., 8.]]),
 array([[2., 4.],
        [4., 8.]]),
 array([[2., 4.],
        [4., 8.]]))

In [2]:
import numpy as np

#
# [9] 회귀계수 추정 
# 
# - (X_T · X)-¹ · X_T · y = β 
# - (p x n) (n x p) (p x n) (n x 1)
#   → (p x p) (p x n) (n x 1)
#   → (p x n) (n x 1)
#   → (p x 1)
# - 그래서 X 에 1 vector 를 제일 앞에 추가해서 상수항도 함께 고려하는 것임!
#
X1 = np.array([1,2,3,4,5]).reshape(-1,1)
X2 = np.array([2,5,3,2,11]).reshape(-1,1)
y = np.array([3,4,5,12,23]).reshape(-1,1)
X = np.concatenate([X1, X2], axis = 1)

# 절편 미고려 時
print(np.dot(np.dot(np.linalg.inv(np.dot(X.T , X)), X.T), y))

# 절편 고려 時
X = np.concatenate([np.ones((5,1)),X] , axis=1)
print(np.dot(np.dot(np.linalg.inv(np.dot(X.T , X)), X.T), y))

# @ 연산자 (행렬 곱)
np.linalg.inv(X.T @ X) @ X.T @ y

X


[[2.14510215]
 [0.84546883]]
[[-5.08587896]
 [ 3.51181556]
 [ 0.85878963]]


array([[ 1.,  1.,  2.],
       [ 1.,  2.,  5.],
       [ 1.,  3.,  3.],
       [ 1.,  4.,  2.],
       [ 1.,  5., 11.]])

In [84]:
#
# [10] 행렬식 (Determinant) 과 역행렬
# 
# det(A) = 대각 행렬들의 합의 차 
# - det(A)= 0    : 역행렬이 존재할 수 없음 → 선형 변환 時 일부 Vector 가 다른 Vector 로 매핑되어 손실이 발생함을 의미
# - |det(A)| > 0 : 스케일링 효과를 갖는 행렬
#
# A · A-¹ = I   * I 는 항등행렬 (대각 행렬이 모두 1, 나머지는 0)
#
#   
import numpy as np

X1 = np.array([1,2,3])
X2 = np.array([2,5,3])
X3 = np.array([3,4,5])
X = np.stack([X1, X2, X2], axis = 1)

# 행렬식과 역행렬
np.linalg.det(X), np.linalg.inv(X)

(-4.99600361081321e-16,
 array([[ 0.00000000e+00,  3.33333333e-01,  6.66666667e-01],
        [-1.80143985e+16,  6.00479950e+15,  2.00159983e+15],
        [ 1.80143985e+16, -6.00479950e+15, -2.00159983e+15]]))

In [87]:
#
# [11] 고유벡터(eigen vector) 와 고유값(eigen value)  
#
# A · x = λ · x 가 존재할 때 λ 를 고유값, x 를 고유벡터  * A 는 정방행렬
# 
# - 고유벡터 :  선형 변환 時 방향을 보존하고 크기만 변하는 벡터 (x)
# - 고유값   :  고유벡터(x)가 선형 변환에 대해서 얼마나 변화되는지를 나타내는 값 (λ)
# - 고유값과 고유벡터 개수 = 변수 개수
# 
# → A·x - λ·x = 0   , 0 벡터가 존재한다고 가정하고 x, λ 를 구함
# → (A - λ·I)·x = 0 , 0 벡터가 존재하려면 det(A - λ) = 0 이여야 함 (otherwise, x 가 0 벡터야함)
# → det(A - λ·I) = 0 을 만족하는 λ 를 구하고 (A - λ·I)·x = 0 을 만족하는 x 벡터를 구함
#
import numpy as np

X1 = np.array([1,2,3])
X2 = np.array([2,5,3])
X3 = np.array([3,4,5])
X = np.stack([X1, X2, X2], axis = 1)

eigen_vals, eigen_vectors = np.linalg.eig(X)
eigen_vals, eigen_vectors

(array([ 9.21699057e+00, -2.16990566e-01, -8.40763765e-16]),
 array([[-3.20315142e-01, -1.36872590e-01, -1.90430373e-16],
        [-7.83411514e-01, -6.57569819e-01, -7.07106781e-01],
        [-5.32601736e-01,  7.40856145e-01,  7.07106781e-01]]))

In [127]:
#
# [12] 주성분 (Pinciple Components)
# 

import numpy as np

# 임의 데이터
X1 = np.array([2,0,1])
X2 = np.array([0,1,1])
X = np.stack([X1, X2], axis=1)

#
# 방법1 : Numpy 기본 함수로 구하기  
#

# (1) 표준화 (Standard Scaling)
Z = (X - np.mean(X,axis = 0)) / np.std(X, axis = 0)
# # [ 1.22474487 -1.41421356]
# # [-1.22474487  0.70710678]
# # [ 0.          0.70710678]

# (2) 공분산 행렬
C = np.cov(Z.T, ddof=0)
# [ 1.       , -0.8660254],
# [-0.8660254,  1.       ]

# (3) 고유벡터, 고유값
e_vals, e_vecs = np.linalg.eig(C)
# λ1 ,λ2 = [1.8660254, 0.1339746]
# x1 = [ 0.70710678,  0.70710678],
# x2 = [-0.70710678,  0.70710678]

# (4) 주성분
PC1 = Z @ e_vecs[0] # [-0.1339746, -0.3660254,  0.5 ]
PC2 = Z @ e_vecs[1] # [-1.8660254,  1.3660254,  0.5 ]

PC1, PC2

(array([-0.1339746, -0.3660254,  0.5      ]),
 array([-1.8660254,  1.3660254,  0.5      ]))

In [133]:
import numpy as np
from sklearn.decomposition import PCA

X1 = np.array([2, 0, 1])
X2 = np.array([0, 1, 1])
X = np.stack([X1, X2], axis=0)
Z = (X - np.mean(X, axis=1).reshape(-1, 1)) / np.std(X, axis=1).reshape(-1, 1)

pca = PCA(n_components=2)
pca.fit(Z)

pca.components_
# PC_scores = pca.transform(Z)
# print("주성분 점수:")
# print(PC_scores)


array([[-0.78867513,  0.57735027,  0.21132487],
       [-0.61069682, -0.69597283, -0.37771844]])

In [125]:
import numpy as np

X1 = np.array([2, 0, 1])
X2 = np.array([0, 1, 1])
X = np.stack([X1, X2], axis=1)

Z = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
C = np.cov(Z.T, ddof=0)

e_vals, e_vecs = np.linalg.eig(C)

# 고유값을 기준으로 고유 벡터를 정렬
# idx = e_vals.argsort()[::-1]
# e_vals = e_vals[idx]
# e_vecs = e_vecs[:, idx]

PC1_scores = Z @ e_vecs[:, 0]
PC2_scores = Z @ e_vecs[:, 1]

print("PC1 점수:", PC1_scores)
print("PC2 점수:", PC2_scores)


PC1 점수: [ 1.8660254 -1.3660254 -0.5      ]
PC2 점수: [-0.1339746 -0.3660254  0.5      ]


In [ ]:
#
# [13] 표준 오차 (Standard Error)
#
# - 개념 : 추정치의 변동성을 측정한 값으로 추정치에 따라 계산 방식이 다름!!
#
# - 종류 : 모집단에 대한 평균, 비율, 평균의 차이, 비율의 차이, 회귀 계수 등
#  . SEM (Standard Error of the Mean) = S / sqrt(n)                     *  S 표본 표준 편차, n 표본 크기
#  . SEP (Standard Error of the Proportion) = sqrt(p * (1 - p) / n)     *  p 표본 비율, n 표본 크기
#  . SERC(Standard Error of the Regression Coefficient) = sqrt[ { (Σ(실제값 - 예측값)² / (n - k - 1) } / {Σ(Xi - X_mean)²} ]
#    *  n 데이터 개수, k는 독립변수(예측 변수) 개수입니다(절편은 제외) , Xi 각 독립 변수의 값, X_mean 독립 변수의 평균
#  . SEDM(Standard Error of the Difference Between Means) = sqrt((S1² / n1) + (S2² / n2))
#    *  S1과 S2는 각 표본의 표준 편차, n1과 n2는 각 표본의 크기
#  . SEDP(Standard Error of the Difference Between Proportions)  = sqrt((p1 * (1-p1) / n1) + (p2 * (1-p2) / n2))
#    *  p1과 p2는 각 표본 비율, n1과 n2는 각 표본의 크기
#
# - 각 표준 오차는 정규분포(표본 크기에 따라 t-분포)를 따른다고 볼 수 있으나 상황에 따라 다름
#  


In [146]:
#
# [14] 추정치의 신뢰구간
#
# - 개념 : 추정치 주변의 불확실성을 나타내며, 특정 신뢰 수준에서 참 값이 포함될 것으로 예상되는 구간입니다
# 
# - 공식 : "추정치 ± 통계량_α/2 * 표준오차" 
#   → 통계량_α/2" : 해당 신뢰 수준에서의 임계값 (예: 정규분포에서 유의수준 5% 에 해당 하는 통계량은 1.96.(양측) )
#   → 표준오차 : 표본에서 추출되는 과정에서 발생할 수 있는 추정치 변동성
#   →→ 추정치의 불확실성을 고려하여 주어진 신뢰 수준에서 참 값이 포함될 것으로 예상되는 범위를 계산
# 

import numpy as np
import pandas as pd
import scipy.stats as sp
from statsmodels.formula.api import ols

data = pd.read_csv("./data/bike.csv")

model = ols(formula='casual ~ temp', data = data).fit()

coff = model.params['temp'] 
se = model.bse['temp']
t_staticstics_half = np.abs(sp.t.ppf(q=0.25, df = data.shape[0] - 2))

coff - se * t_staticstics_half, coff + se * t_staticstics_half


(2.9584181038785426, 3.0317308635070943)